In [10]:
import pandas as pd
import numpy as np
import altair as alt

import os

In [2]:
dataset_dir = "/home/sam/Desktop/datasets/loan-prediction/"
os.listdir(dataset_dir)

['train.csv', 'sample_submission.csv', 'test.csv']

In [3]:
df = pd.read_csv(dataset_dir + "train.csv")

In [32]:
from sklearn.preprocessing import OneHotEncoder

In [30]:
categorical_data = ['person_home_ownership', 'loan_intent']

In [34]:
df_cats = df[categorical_data]
oh_encoder = OneHotEncoder(categories='auto')
oh_encoder.fit(df_cats)

def set_ohe_cat_columns(df, encoder, cat_cols):
    col_names = encoder.get_feature_names_out()
    
    cat_data = df[cat_cols]
    ohe_values = encoder.transform(cat_data).toarray()
    ohe_df = pd.DataFrame(ohe_values, columns=col_names).astype(int)

    df = pd.concat([df, ohe_df], axis=1)
    return df

df = set_ohe_cat_columns(df, oh_encoder, categorical_data)

In [70]:
grade_mapper = {'G': '0', 'F': '1', 'E': '2', 'D': '3', 'C': '4', 'B': '5', 'A': '6'}
grade_mapper_preserver = {value:key for key, value in grade_mapper.items()}
ordvar = df['loan_grade'].replace(grade_mapper)

df['loan_grade'] = ordvar.astype(int)

In [71]:
default_mapper = {'N': '0', 'Y': '1'}
default_mapper_preserver = {value:key for key, value in default_mapper.items()}

df['cb_person_default_on_file'] = df['cb_person_default_on_file'].replace(default_mapper).astype(int)

In [105]:
irrel_cols = ['person_home_ownership', 'loan_intent', 'person_income', 'cb_person_cred_hist_length']
modelling_df = df.drop(columns=irrel_cols)

In [106]:
from sklearn.model_selection import train_test_split
y = modelling_df['loan_status']
X = modelling_df.drop(columns=['loan_status'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [107]:
df['loan_status'].unique()

array([0, 1])

In [108]:
modelling_df.dtypes

id                                  int64
person_age                          int64
person_emp_length                 float64
loan_grade                          int64
loan_amnt                           int64
loan_int_rate                     float64
loan_percent_income               float64
cb_person_default_on_file           int64
loan_status                         int64
person_home_ownership_MORTGAGE      int64
person_home_ownership_OTHER         int64
person_home_ownership_OWN           int64
person_home_ownership_RENT          int64
loan_intent_DEBTCONSOLIDATION       int64
loan_intent_EDUCATION               int64
loan_intent_HOMEIMPROVEMENT         int64
loan_intent_MEDICAL                 int64
loan_intent_PERSONAL                int64
loan_intent_VENTURE                 int64
dtype: object

In [109]:
from sklearn.ensemble import GradientBoostingClassifier

gbr = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gbr.fit(X_train, y_train) 

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=42)

In [110]:
from sklearn.metrics import roc_auc_score

y_pred = gbr.predict(X_test)
roc_auc_score(y_test, y_pred)

np.float64(0.8131281159445879)